### Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd 'drive/MyDrive/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation/'

/content/drive/.shortcut-targets-by-id/1h_oF16bG32l75GPobgHe_9HgdUduLFxT/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation


### Importing Necessary Libraries


In [3]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import os

### Importing Data
We import the data of a young subject's 3D-Tensor Data of dimensions (TR, Voxel, Number of Subjects) for the 5 different ROIs as follows:
- **Prefrontal areas:** dlPFC, vlPFC, lOFC, dmPFC, mPFC, mOFC
- **Default-mode areas:** mPFC, ACC, PCC, Precuneus, mOFC
- **Sensory areas:** VC
- **Multisensory area:** pSTS
- **Some other areas:** TP, IPL, mCC, Put, PCG, Nac, INS





In [4]:
file_names = [name for name in os.listdir('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name)]
file_names = [name for name in file_names if ("537" not in name)]

In [5]:
ROI_names = [file_name.replace("data_","").replace(".npy","") for file_name in file_names]

In [6]:
data_ori = list()
for file_name in file_names:
  data_ori.append(np.load('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy/'+file_name))

### Data Preparation

Now, here we prepare the data to be fed to the Neural Network Architechtures. We can't predict the time ahead upto which the subjects are predicting correctly so, we will take a bit round about way to predict the BOLD values ahead in time(for several choices of time points e.g. 1 time point ahead, 2 time point ahead, etc) and will compare the results for young and old subjects.

Right from the starting we decide some of the data preparation parameters, so that we can use them as we require.

In [7]:
TRAIN_BATCH_SIZE = 20
TEST_BATCH_SIZE = 10
LOOK_AHEAD = 1
NUM_TEST_SUBS = 10

We will start a basic data preparation where we will average across all the voxels for a ROI and fit the model which predicts this average BOLD value. Later we will avoid doing an average since, all voxels in a ROI is not equally important and hence we will want to predict the BOLD values of several voxels simultaneously.

In [8]:
data = [np.mean(dat, axis=1, keepdims=True) for dat in data_ori]

Let's Start the Data Preparation for lOFC later we will iterate the same procedure for the other ROIs.

In [9]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

lOFC Average BOLD Shape: (189, 1, 111)


Below, we see the average BOLD time series plot for all the ROIs.

We have changed the data in the form (Number of Subjects, TR, Voxels) below.

In [10]:
data = [np.transpose(dat, (2,0,1)) for dat in data]

In [11]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

lOFC Average BOLD Shape: (111, 189, 1)


We split the dataset of 111 young subjects into 101 subjects in the training data and 10 subject in the test data.

In [12]:
train_data_unscaled = [dat[:-NUM_TEST_SUBS] for dat in data]
test_data_unscaled = [dat[-NUM_TEST_SUBS:] for dat in data]  

We scaled the BOLD values of each voxel in the train data between 0 and 1 for each voxel. We use the same MinMaxScaler fit on the train data on the test data.

In [13]:
scalers = list()
for i in range(len(ROI_names)):
  scalers.append(MinMaxScaler(feature_range = (0, 1)))
train_data = [sc.fit_transform(dat[:,:,0]) for sc, dat in zip(scalers, train_data_unscaled)] 
test_data = [sc.transform(dat[:,:,0]) for sc, dat in zip(scalers, test_data_unscaled)]

In [14]:
train_data = [np.expand_dims(dat, axis = -1) for dat in train_data]
test_data = [np.expand_dims(dat, axis = -1) for dat in test_data]

In [15]:
print(ROI_names[-1], "Train Data Shape: ", train_data[-1].shape)
print(ROI_names[-1], "Test Data Shape: ", test_data[-1].shape)

lOFC Train Data Shape:  (101, 189, 1)
lOFC Test Data Shape:  (10, 189, 1)


We now build a data generator which would produce mini-batches during the training of the neural network.

In [16]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_TRs, batch_size, look_ahead=1):
        self.data = data
        self.num_TRs = num_TRs
        self.batch_size = batch_size
        self.current_idx = 0
        self.look_ahead = look_ahead

    def generate(self):
        while True:
            if self.current_idx + self.batch_size >= self.data.shape[0]:
              self.current_idx = 0
            x = self.data[self.current_idx:(self.current_idx+self.batch_size),:(-self.look_ahead),:]
            y = self.data[self.current_idx:(self.current_idx+self.batch_size),self.look_ahead:,:]
            self.current_idx = self.current_idx + self.batch_size
            yield x, y 

In [17]:
train_data_generator = [KerasBatchGenerator(dat, dat.shape[1], TRAIN_BATCH_SIZE, LOOK_AHEAD) for dat in train_data]
test_data_generator = [KerasBatchGenerator(dat, dat.shape[1], TEST_BATCH_SIZE, LOOK_AHEAD) for dat in test_data]

In [18]:
batch_x, batch_y = next(train_data_generator[-1].generate())
print(ROI_names[-1], "Train Batch X Shape: ", batch_x.shape)
print(ROI_names[-1], "Train Batch y Shape: ", batch_y.shape)

lOFC Train Batch X Shape:  (20, 188, 1)
lOFC Train Batch y Shape:  (20, 188, 1)


In [19]:
batch_x, batch_y = next(test_data_generator[-1].generate())
print(ROI_names[-1], "Test Batch X Shape: ", batch_x.shape)
print(ROI_names[-1], "Test Batch y Shape: ", batch_y.shape)

lOFC Test Batch X Shape:  (10, 188, 1)
lOFC Test Batch y Shape:  (10, 188, 1)


#### Model-1

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 1`
- `hidden units = 8`
- `dropout = 0.3`

In [20]:
HIDDEN_UNITS = 8
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 1

In [21]:
def avg_bold_model():
  model = Sequential()
  for _ in range(LSTM_LAYERS):
    model.add(LSTM(HIDDEN_UNITS, return_sequences=True))
  model.add(Dropout(0.3))
  for _ in range(DENSE_LAYERS-1):
    model.add(TimeDistributed(Dense(HIDDEN_UNITS)))
  model.add(TimeDistributed(Dense(1)))
  return model

In [22]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (10, 188, 8)              320       
                                                                 
 dropout (Dropout)           (10, 188, 8)              0         
                                                                 
 time_distributed (TimeDistr  (10, 188, 1)             9         
 ibuted)                                                         
                                                                 
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [23]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [24]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [25]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [27]:
a_TRs = np.array([56, 67, 73, 81, 134, 142, 155, 167, 174, 180])
d_TRs = np.array([67, 72, 85, 89, 145, 154, 168, 174, 180, 188])
and_TRs = np.union1d(a_TRs, d_TRs)

In [28]:
def generate_each_sub_n_overall_pred_corr(preds):
  
  print("***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***")
  print()
  avg_r = list()
  for i in range(len(ROI_names)):
    df = pd.DataFrame({"Subs":[], "A1":[], "A2":[], "A3":[], "A4":[], "A5":[], "D1":[], "D2":[], "D3":[], "D4":[], "D5":[], "Avg_Asc":[], "Avg_Des":[], "Overall":[]})
    actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
    pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[i][:,:,0], preds[i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
    for j in range(NUM_TEST_SUBS):
      df_dict = dict()
      df_dict['Subs'] = j+1
      df_dict['Overall'] = np.round(np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1], 2)
      sum_as = 0 # stores the sum of each the subject's avg BOLD 5 ascents correlation between actual and preds 
      sum_des = 0 # stores the sum of each the subject's avg BOLD 5 descents correlation between actual and preds 
      for k in range(5):
        a = np.round(np.corrcoef(actual_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]], pred_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]])[0,1], 2)
        d = np.round(np.corrcoef(actual_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]], pred_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]])[0,1], 2)
        df_dict['A'+str(k+1)] = a
        df_dict['D'+str(k+1)] = d
        sum_as += a
        sum_des += d
      df_dict['Avg_Asc'] = np.round(sum_as/5, 2)
      df_dict['Avg_Des'] = np.round(sum_des/5, 2)
      df = df.append(df_dict, ignore_index = True)
    print()
    print(f"***{ROI_names[i]}***")
    display(df)
    path = f"./Model-Inference/YOUNG/LSTM/AVG-BOLD/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-Each-Test-Sub-Corr.xlsx"
    if os.path.isfile(path):  
      with pd.ExcelWriter(path, mode='a') as writer:
        df.to_excel(writer, sheet_name= ROI_names[i])
    else:
      with pd.ExcelWriter(path) as writer:
        df.to_excel(writer, sheet_name= ROI_names[i])



  print("***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***")
  print()
  df = pd.DataFrame({"ROIs":[], "A1":[], "A2":[], "A3":[], "A4":[], "A5":[], "D1":[], "D2":[], "D3":[], "D4":[], "D5":[], "Avg_Asc":[], "Avg_Des":[], "Overall":[]})
  for i in range(len(ROI_names)):
    actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
    pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[i][:,:,0], preds[i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
    sum_r = 0 # stores the sum of all the subjects avg BOLD correlation between actual and preds
    sum_a = np.zeros(5) # stores the sum of all the subjects avg BOLD 5 ascents correlation between actual and preds
    sum_d = np.zeros(5) # stores the sum of all the subjects avg BOLD 5 descents between actual and preds
    for j in range(NUM_TEST_SUBS):
      sum_r += np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1]
      for k in range(5):
        sum_a[k] += np.corrcoef(actual_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]], pred_test_data_output[j,a_TRs[2*k]:a_TRs[2*k+1]])[0,1]
        sum_d[k] += np.corrcoef(actual_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]], pred_test_data_output[j,d_TRs[2*k]:d_TRs[2*k+1]])[0,1]
    df.loc[len(df.index)] = [ROI_names[i], *np.round(sum_a / NUM_TEST_SUBS, 2), *np.round(sum_d / NUM_TEST_SUBS, 2), np.round(np.mean(sum_a / NUM_TEST_SUBS), 2), np.round(np.mean(sum_d / NUM_TEST_SUBS), 2), np.round(sum_r / NUM_TEST_SUBS, 2)]
  display(df)
  path = f"./Model-Inference/YOUNG/LSTM/AVG-BOLD/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-All-ROI-Corr-Avg-Across-Test-Subs.xlsx"
  if os.path.isfile(path):  
    with pd.ExcelWriter(path, mode='a') as writer:
      df.to_excel(writer, sheet_name= ROI_names[i])
  else:
    with pd.ExcelWriter(path) as writer:
      df.to_excel(writer, sheet_name= ROI_names[i])

In [29]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.10,-0.14,0.44,-0.13,0.03,0.04,0.43,0.12,-0.10,0.59,0.06,0.22,0.16
1,2.0,0.62,-0.38,-0.46,0.25,-0.18,-0.58,-0.46,0.05,0.66,0.26,-0.03,-0.01,0.34
2,3.0,0.43,-0.63,-0.82,-0.11,-0.38,-0.50,-0.66,-0.54,-0.69,0.02,-0.30,-0.47,0.36
3,4.0,-0.72,-0.25,-0.48,0.03,-0.23,-0.28,-0.68,-0.38,0.48,-0.40,-0.33,-0.25,0.20
4,5.0,-0.33,-0.65,-0.39,0.35,-0.30,0.11,0.67,0.22,-0.12,-0.84,-0.26,0.01,0.21
5,6.0,-0.25,-0.47,0.27,-0.17,-0.17,-0.76,-0.39,-0.54,0.64,0.07,-0.16,-0.20,0.36
6,7.0,0.51,-0.45,0.29,0.07,0.09,0.85,-0.92,-0.76,0.12,0.52,0.10,-0.04,0.11
7,8.0,0.33,-0.54,-0.20,0.06,0.24,0.27,-0.30,0.29,0.17,0.12,-0.02,0.11,0.35
8,9.0,-0.13,0.25,-0.36,-0.03,-0.28,-0.05,-0.48,-0.16,-0.24,0.09,-0.11,-0.17,0.27
9,10.0,0.42,-0.67,0.57,0.36,-0.76,0.70,-0.60,-0.43,-0.13,-0.53,-0.02,-0.20,0.06



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.18,0.12,-0.58,0.44,-0.07,0.18,-0.44,0.19,-0.29,-0.67,0.02,-0.21,0.23
1,2.0,-0.18,-0.03,-0.19,0.01,0.09,-0.59,-0.53,0.34,-0.08,-0.23,-0.06,-0.22,0.51
2,3.0,0.38,-0.63,-0.32,0.22,0.59,0.60,0.84,-0.52,-0.28,0.40,0.05,0.21,0.15
3,4.0,-0.29,-0.10,-0.42,-0.01,0.57,0.40,0.33,0.11,-0.43,0.05,-0.05,0.09,0.45
4,5.0,0.22,0.54,-0.64,0.62,0.62,0.63,-0.77,0.73,-0.26,-0.65,0.27,-0.06,0.34
5,6.0,-0.13,-0.59,-0.28,-0.02,0.24,0.91,0.13,0.27,0.43,-0.67,-0.16,0.21,0.40
6,7.0,-0.16,-0.48,-0.77,0.46,-0.27,0.49,0.01,-0.18,-0.47,-0.48,-0.24,-0.13,0.34
7,8.0,0.38,-0.47,-0.18,-0.75,0.00,0.21,-0.23,0.32,0.48,-0.17,-0.20,0.12,0.53
8,9.0,-0.30,0.72,-0.19,0.78,0.20,-0.65,-0.50,-0.37,0.54,0.01,0.24,-0.19,0.19
9,10.0,-0.57,-0.36,-0.28,0.35,0.53,-0.05,-0.31,0.05,0.11,-0.37,-0.07,-0.11,0.30



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.24,-0.22,0.56,-0.41,-0.35,0.89,0.54,0.07,-0.48,-0.06,-0.04,0.19,0.14
1,2.0,0.51,0.44,-0.43,-0.09,-0.49,-0.90,-0.22,0.04,-0.17,-0.54,-0.01,-0.36,0.09
2,3.0,0.65,-0.68,-0.46,0.22,-0.02,0.77,0.28,-0.51,-0.03,0.42,-0.06,0.19,0.49
3,4.0,-0.28,-0.18,0.39,-0.09,0.39,0.93,-0.56,0.27,0.30,0.66,0.05,0.32,0.43
4,5.0,-0.45,-0.11,0.31,0.16,0.32,0.28,0.69,0.46,0.06,-0.27,0.05,0.24,0.22
5,6.0,-0.36,-0.07,0.37,0.01,0.01,0.40,-0.59,0.17,0.27,-0.81,-0.01,-0.11,0.09
6,7.0,0.31,0.20,0.70,-0.08,0.27,0.41,0.01,0.14,0.78,-0.11,0.28,0.25,0.16
7,8.0,-0.14,-0.06,0.70,-0.18,-0.20,0.95,0.43,0.75,0.67,0.31,0.02,0.62,0.65
8,9.0,0.15,-0.45,0.20,-0.45,-0.15,-0.78,-0.79,-0.42,-0.21,-0.97,-0.14,-0.63,0.07
9,10.0,0.67,0.55,0.83,0.66,-0.08,0.83,-0.85,0.30,0.50,0.55,0.53,0.27,-0.10



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.51,-0.54,-0.44,-0.40,-0.18,0.57,0.79,0.36,-0.64,-0.05,-0.21,0.21,0.46
1,2.0,0.53,-0.17,-0.43,-0.34,-0.58,-0.43,0.07,0.74,-0.85,0.16,-0.20,-0.06,0.16
2,3.0,-0.43,0.37,0.14,0.53,-0.38,-0.07,-0.99,0.24,0.23,-0.25,0.05,-0.17,0.14
3,4.0,-0.41,0.16,0.42,0.39,0.31,0.22,-0.16,-0.01,0.30,0.02,0.17,0.07,0.61
4,5.0,-0.63,-0.43,-0.81,0.26,0.23,-0.28,-0.92,-0.09,0.11,0.66,-0.28,-0.10,0.17
5,6.0,0.53,-0.03,-0.14,-0.00,-0.25,0.48,-0.29,-0.37,-0.37,0.19,0.02,-0.07,-0.05
6,7.0,0.06,-0.21,-0.40,0.58,0.36,0.63,-0.20,-0.22,0.00,-0.19,0.08,0.00,0.11
7,8.0,0.03,0.35,0.29,0.30,-0.61,0.53,-0.06,-0.05,-0.22,-0.31,0.07,-0.02,0.62
8,9.0,0.22,0.85,-0.64,-0.34,0.28,0.23,0.34,-0.35,0.50,-0.85,0.07,-0.03,0.44
9,10.0,0.37,0.62,0.01,0.64,-0.02,-0.55,-0.97,0.41,0.26,-0.13,0.32,-0.20,-0.18


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.10,-0.39,-0.11,0.07,-0.19,-0.02,-0.34,-0.21,0.08,-0.01,-0.11,-0.10,0.24
1,ACC,-0.05,-0.13,-0.39,0.21,0.25,0.21,-0.15,0.09,-0.02,-0.28,-0.02,-0.03,0.34
2,AMY,-0.18,-0.11,0.23,0.10,-0.06,0.22,-0.08,-0.06,-0.13,-0.03,-0.00,-0.01,0.33
3,Cau,-0.02,0.28,-0.02,0.11,0.11,-0.00,0.18,0.16,-0.36,0.20,0.09,0.04,0.21
4,dlPFC,-0.09,-0.09,0.13,0.12,-0.10,-0.06,0.66,-0.03,-0.08,0.07,-0.01,0.11,0.27
5,INS,0.25,-0.03,0.64,-0.00,-0.03,-0.01,0.02,0.08,0.25,0.22,0.17,0.11,0.28
6,IPL,0.33,0.08,0.27,-0.15,0.23,0.41,0.85,0.11,-0.30,-0.10,0.15,0.19,0.28
7,mCC,-0.08,-0.03,-0.13,0.02,-0.00,-0.38,-0.15,-0.13,0.23,0.00,-0.04,-0.08,0.23
8,Nac,0.09,0.27,0.11,0.13,0.14,-0.05,0.01,0.14,-0.35,0.07,0.15,-0.04,0.30
9,PCC,-0.33,-0.28,-0.01,0.17,0.48,0.39,-0.18,0.03,0.10,-0.14,0.01,0.04,0.31


#### Model-2

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 1`
- `hidden units = 16`
- `dropout = 0.3`

In [30]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 1

In [31]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (10, 188, 16)             1152      
                                                                 
 dropout_21 (Dropout)        (10, 188, 16)             0         
                                                                 
 time_distributed_21 (TimeDi  (10, 188, 1)             17        
 stributed)                                                      
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [32]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [ ]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [34]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-0{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [35]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [36]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.10,-0.19,0.48,0.26,-0.02,-0.04,0.36,-0.35,0.30,0.48,0.13,0.15,0.22
1,2.0,0.59,-0.30,-0.52,0.15,-0.15,-0.76,-0.54,-0.07,0.53,0.16,-0.05,-0.14,0.42
2,3.0,0.58,-0.46,-0.17,0.30,0.10,-0.47,-0.59,0.42,-0.65,0.19,0.07,-0.22,0.60
3,4.0,-0.35,-0.13,-0.37,-0.07,-0.18,-0.29,-0.69,-0.29,0.51,-0.36,-0.22,-0.22,0.30
4,5.0,-0.27,-0.85,-0.24,0.38,-0.07,0.07,0.61,-0.03,0.13,-0.80,-0.21,-0.00,0.27
5,6.0,-0.39,-0.44,0.18,-0.23,-0.15,-0.67,-0.41,-0.58,0.62,-0.10,-0.21,-0.23,0.38
6,7.0,0.71,-0.50,0.26,0.15,0.29,0.80,-0.96,-0.75,0.00,0.39,0.18,-0.10,0.17
7,8.0,0.50,-0.48,-0.13,0.15,0.13,0.28,-0.39,0.16,0.06,0.17,0.03,0.06,0.50
8,9.0,0.20,0.12,-0.12,0.04,-0.25,-0.20,-0.49,0.43,-0.72,0.37,-0.00,-0.12,0.34
9,10.0,0.51,-0.62,0.54,0.34,-0.68,0.78,-0.65,-0.45,-0.07,-0.41,0.02,-0.16,0.06



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.16,0.06,-0.50,0.19,-0.07,0.01,-0.32,0.15,-0.18,-0.73,-0.03,-0.21,0.19
1,2.0,-0.18,-0.04,-0.42,-0.06,0.06,-0.67,-0.53,0.32,-0.05,-0.17,-0.13,-0.22,0.42
2,3.0,0.41,-0.45,-0.54,0.06,0.50,0.59,0.73,-0.46,-0.33,0.52,-0.00,0.21,0.45
3,4.0,-0.10,-0.11,-0.11,-0.06,0.58,0.42,0.25,0.32,-0.45,0.07,0.04,0.12,0.40
4,5.0,0.08,0.53,-0.79,0.59,0.60,0.58,-0.81,0.69,-0.34,-0.73,0.20,-0.12,0.28
5,6.0,-0.16,-0.63,-0.18,-0.03,0.18,0.89,0.06,0.25,0.48,-0.66,-0.16,0.20,0.32
6,7.0,-0.16,-0.45,-0.56,0.35,-0.32,0.38,-0.04,-0.14,-0.40,-0.48,-0.23,-0.14,0.31
7,8.0,0.28,-0.49,-0.25,-0.89,0.02,0.27,-0.28,0.41,0.42,-0.12,-0.27,0.14,0.53
8,9.0,-0.18,0.67,-0.34,0.69,0.06,-0.75,-0.85,-0.26,0.38,-0.17,0.18,-0.33,0.35
9,10.0,-0.52,-0.37,-0.35,0.33,0.48,-0.16,-0.27,0.04,0.07,-0.42,-0.09,-0.15,0.31



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.17,-0.19,0.56,0.03,-0.32,0.88,0.57,0.12,-0.47,-0.03,0.05,0.21,0.25
1,2.0,0.51,0.45,-0.33,-0.06,-0.53,-0.91,-0.22,0.03,-0.27,-0.52,0.01,-0.38,0.18
2,3.0,0.72,-0.47,-0.46,0.14,-0.06,0.77,0.39,-0.29,0.12,0.43,-0.03,0.28,0.43
3,4.0,-0.29,-0.18,0.37,-0.08,0.41,0.92,-0.61,0.27,0.25,0.67,0.05,0.30,0.63
4,5.0,-0.38,-0.09,0.42,0.17,0.37,0.19,0.82,0.51,0.10,-0.26,0.10,0.27,0.23
5,6.0,-0.39,-0.06,0.35,0.01,-0.06,0.38,-0.58,0.14,0.25,-0.73,-0.03,-0.11,0.17
6,7.0,0.37,0.14,0.63,0.06,0.26,0.36,-0.05,0.11,0.76,-0.14,0.29,0.21,0.22
7,8.0,-0.03,-0.10,0.69,-0.17,-0.17,0.95,0.46,0.72,0.64,0.26,0.04,0.61,0.73
8,9.0,0.08,-0.24,0.25,-0.18,-0.18,-0.83,-0.71,-0.19,0.01,-0.96,-0.05,-0.54,0.17
9,10.0,0.70,0.54,0.82,0.64,-0.02,0.84,-0.79,0.35,0.50,0.57,0.54,0.29,-0.00



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.50,-0.61,-0.41,-0.32,-0.24,0.57,0.76,0.32,-0.63,-0.01,-0.22,0.20,0.38
1,2.0,0.55,-0.18,-0.53,-0.35,-0.54,-0.41,0.03,0.72,-0.82,0.12,-0.21,-0.07,0.17
2,3.0,-0.44,0.31,0.04,0.53,-0.18,-0.04,-0.99,0.34,0.16,-0.15,0.05,-0.14,0.52
3,4.0,-0.34,0.21,0.27,0.54,-0.06,0.19,-0.09,-0.03,0.41,-0.02,0.12,0.09,0.79
4,5.0,-0.59,-0.27,-0.80,0.31,0.31,-0.34,-0.93,-0.07,0.10,0.66,-0.21,-0.12,0.15
5,6.0,0.53,-0.04,-0.13,0.03,-0.24,0.48,-0.37,-0.55,-0.33,0.13,0.03,-0.13,0.07
6,7.0,0.04,-0.26,-0.32,0.58,0.40,0.59,-0.26,-0.25,-0.01,-0.16,0.09,-0.02,0.14
7,8.0,0.03,0.10,0.33,0.26,-0.69,0.57,-0.08,0.04,-0.24,-0.32,0.01,-0.01,0.65
8,9.0,0.19,0.86,-0.68,-0.34,0.34,0.59,0.16,-0.44,0.54,-0.81,0.07,0.01,0.63
9,10.0,0.37,0.11,0.02,0.63,-0.06,-0.55,-0.99,0.39,0.28,-0.09,0.21,-0.19,-0.13


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.22,-0.38,-0.01,0.15,-0.10,-0.05,-0.37,-0.15,0.07,0.01,-0.03,-0.10,0.33
1,ACC,-0.04,-0.13,-0.40,0.12,0.21,0.16,-0.21,0.13,-0.04,-0.29,-0.05,-0.05,0.36
2,AMY,-0.14,-0.12,0.26,0.08,-0.09,0.11,-0.15,-0.06,-0.24,-0.05,-0.00,-0.07,0.39
3,Cau,-0.05,0.24,0.00,0.07,0.10,-0.02,0.15,0.15,-0.36,0.14,0.07,0.01,0.22
4,dlPFC,-0.06,-0.12,0.20,0.08,-0.11,-0.14,0.49,-0.00,-0.10,0.07,-0.00,0.07,0.29
5,INS,0.29,-0.06,0.62,0.03,-0.05,-0.02,-0.01,0.06,0.24,0.20,0.17,0.09,0.27
6,IPL,0.32,-0.02,0.18,-0.07,0.10,0.44,0.84,0.23,-0.25,-0.17,0.10,0.22,0.35
7,mCC,0.01,-0.16,-0.14,0.09,-0.02,-0.33,-0.20,-0.09,0.14,-0.06,-0.04,-0.11,0.33
8,Nac,0.03,0.29,0.06,0.09,0.16,-0.06,0.07,0.12,-0.38,0.07,0.13,-0.04,0.25
9,PCC,-0.26,-0.21,-0.07,0.29,0.48,0.42,-0.25,0.17,0.08,-0.04,0.05,0.08,0.41


#### Model-3

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 1`
- `hidden units = 32`
- `dropout = 0.3`

In [37]:
HIDDEN_UNITS = 32
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 1

In [38]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_42 (LSTM)              (10, 188, 32)             4352      
                                                                 
 dropout_42 (Dropout)        (10, 188, 32)             0         
                                                                 
 time_distributed_42 (TimeDi  (10, 188, 1)             33        
 stributed)                                                      
                                                                 
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [39]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [40]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [41]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [43]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.12,-0.18,0.52,-0.13,-0.04,0.05,0.40,0.24,0.05,0.49,0.06,0.25,0.16
1,2.0,0.56,-0.39,-0.41,0.20,-0.19,-0.56,-0.41,0.09,0.72,0.23,-0.05,0.01,0.36
2,3.0,0.49,-0.63,-0.78,-0.11,-0.08,-0.47,-0.63,-0.39,-0.78,0.07,-0.22,-0.44,0.48
3,4.0,-0.55,-0.24,-0.46,0.00,-0.22,-0.17,-0.65,-0.24,0.48,-0.41,-0.29,-0.20,0.27
4,5.0,-0.43,-0.72,-0.38,0.40,-0.19,0.17,0.65,0.23,-0.08,-0.90,-0.26,0.01,0.22
5,6.0,-0.32,-0.51,0.22,-0.20,-0.18,-0.77,-0.38,-0.59,0.65,0.03,-0.20,-0.21,0.37
6,7.0,0.63,-0.44,0.34,-0.02,0.14,0.80,-0.91,-0.75,0.16,0.57,0.13,-0.03,0.15
7,8.0,0.32,-0.54,-0.16,0.10,0.22,0.26,-0.30,0.20,0.05,0.14,-0.01,0.07,0.41
8,9.0,0.03,0.14,-0.29,-0.07,-0.29,0.23,-0.48,-0.18,-0.40,0.15,-0.10,-0.14,0.29
9,10.0,0.44,-0.63,0.54,0.35,-0.76,0.61,-0.57,-0.51,-0.10,-0.49,-0.01,-0.21,0.03



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.16,-0.08,-0.53,-0.19,-0.06,-0.09,-0.47,0.16,-0.20,-0.66,-0.14,-0.25,0.23
1,2.0,-0.32,0.02,-0.56,-0.05,0.12,-0.59,-0.41,0.35,0.04,-0.23,-0.16,-0.17,0.46
2,3.0,0.11,-0.68,-0.41,0.01,0.54,0.63,0.81,-0.47,-0.51,0.46,-0.09,0.18,0.37
3,4.0,-0.08,-0.13,-0.14,0.00,0.54,0.33,0.41,0.45,-0.38,-0.06,0.04,0.15,0.39
4,5.0,0.19,0.37,-0.80,0.57,0.64,0.61,-0.88,0.74,-0.36,-0.72,0.19,-0.12,0.08
5,6.0,-0.15,-0.64,-0.13,-0.01,0.25,0.87,0.20,0.16,0.48,-0.67,-0.14,0.21,0.32
6,7.0,-0.11,-0.49,-0.57,0.23,-0.23,0.51,0.03,0.05,-0.32,-0.43,-0.23,-0.03,0.29
7,8.0,0.35,-0.59,-0.09,-0.86,0.02,0.21,-0.30,0.48,0.49,-0.14,-0.23,0.15,0.56
8,9.0,-0.33,0.67,-0.38,0.56,0.09,-0.45,-0.55,-0.46,0.36,-0.20,0.12,-0.26,0.24
9,10.0,-0.50,-0.53,-0.40,0.36,0.51,-0.33,-0.56,0.07,0.06,-0.40,-0.11,-0.23,0.28



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.27,-0.24,0.56,-0.52,-0.36,0.90,0.53,0.03,-0.48,-0.02,-0.06,0.19,0.16
1,2.0,0.50,0.46,-0.52,-0.09,-0.46,-0.88,-0.23,0.05,-0.16,-0.58,-0.02,-0.36,0.04
2,3.0,0.57,-0.82,-0.49,0.18,0.07,0.79,0.34,-0.66,-0.18,0.41,-0.10,0.14,0.50
3,4.0,-0.28,-0.15,0.41,-0.10,0.38,0.94,-0.54,0.23,0.34,0.67,0.05,0.33,0.28
4,5.0,-0.47,-0.13,0.29,0.17,0.33,0.30,0.69,0.40,0.04,-0.30,0.04,0.23,0.19
5,6.0,-0.34,-0.08,0.40,-0.00,0.07,0.42,-0.59,0.20,0.28,-0.84,0.01,-0.11,-0.00
6,7.0,0.27,0.22,0.73,-0.14,0.30,0.45,0.06,0.14,0.80,-0.11,0.28,0.27,0.15
7,8.0,-0.15,-0.21,0.65,-0.14,-0.21,0.95,0.37,0.74,0.66,0.28,-0.01,0.60,0.65
8,9.0,0.18,-0.59,0.18,-0.55,-0.14,-0.74,-0.85,-0.51,-0.23,-0.92,-0.18,-0.65,-0.01
9,10.0,0.67,0.53,0.81,0.64,-0.06,0.84,-0.89,0.23,0.47,0.54,0.52,0.24,-0.12



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.43,-0.59,-0.30,-0.02,-0.44,0.49,0.75,0.14,-0.57,0.02,-0.18,0.17,0.45
1,2.0,0.55,-0.24,-0.63,-0.50,-0.17,-0.61,0.11,0.65,-0.72,0.09,-0.20,-0.10,0.19
2,3.0,-0.05,0.30,-0.05,0.59,-0.19,-0.22,-0.94,0.61,0.49,0.34,0.12,0.06,0.61
3,4.0,-0.30,0.34,0.12,0.63,0.01,0.18,-0.07,0.01,0.42,-0.02,0.16,0.10,0.81
4,5.0,-0.54,-0.08,-0.70,0.35,0.36,-0.62,-0.84,-0.20,0.16,0.55,-0.12,-0.19,0.24
5,6.0,0.54,-0.05,-0.28,0.08,-0.25,0.32,-0.57,-0.56,-0.33,0.06,0.01,-0.22,0.05
6,7.0,-0.02,-0.35,-0.40,0.60,0.40,0.51,-0.42,-0.30,-0.13,-0.14,0.05,-0.10,0.15
7,8.0,0.30,-0.21,0.25,0.35,-0.79,0.47,-0.24,-0.01,-0.31,-0.43,-0.02,-0.10,0.77
8,9.0,0.30,0.85,-0.44,-0.22,0.15,0.54,-0.05,-0.43,0.65,-0.59,0.13,0.02,0.74
9,10.0,0.40,0.49,0.07,0.55,-0.17,-0.39,-0.95,0.26,0.37,-0.11,0.27,-0.16,-0.02


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.13,-0.41,-0.08,0.05,-0.16,0.02,-0.33,-0.19,0.08,-0.01,-0.10,-0.09,0.27
1,ACC,-0.07,-0.21,-0.40,0.06,0.24,0.17,-0.17,0.15,-0.03,-0.30,-0.07,-0.04,0.32
2,AMY,-0.15,-0.13,0.26,0.07,-0.09,0.14,-0.13,-0.06,-0.24,-0.06,-0.01,-0.07,0.34
3,Cau,0.06,0.08,0.09,0.07,0.03,-0.01,0.04,0.13,-0.32,0.16,0.07,-0.00,0.27
4,dlPFC,-0.09,-0.18,0.17,0.02,-0.13,-0.08,0.55,-0.07,-0.10,0.05,-0.04,0.07,0.25
5,INS,0.29,-0.08,0.62,0.03,-0.05,-0.02,-0.02,0.05,0.23,0.19,0.16,0.09,0.27
6,IPL,0.33,0.02,0.20,-0.08,0.12,0.43,0.84,0.20,-0.26,-0.15,0.12,0.21,0.34
7,mCC,-0.03,-0.15,-0.10,-0.00,0.01,-0.38,-0.21,-0.12,0.23,-0.04,-0.05,-0.10,0.24
8,Nac,0.06,0.26,0.11,0.08,0.16,-0.07,0.05,0.10,-0.38,0.02,0.13,-0.05,0.25
9,PCC,-0.29,-0.26,-0.06,0.26,0.48,0.40,-0.23,0.13,0.07,-0.08,0.03,0.06,0.35


#### Model-4

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 2`
- `Number of Dense Layers = 1`
- `hidden units = 16`
- `dropout = 0.3`

In [44]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 2
DENSE_LAYERS = 1

In [45]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_63 (LSTM)              (10, 188, 16)             1152      
                                                                 
 lstm_64 (LSTM)              (10, 188, 16)             2112      
                                                                 
 dropout_63 (Dropout)        (10, 188, 16)             0         
                                                                 
 time_distributed_63 (TimeDi  (10, 188, 1)             17        
 stributed)                                                      
                                                                 
Total params: 3,281
Trainable params: 3,281
Non-trainable params: 0
_________________________________________________________________


In [46]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [47]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [48]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [50]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.11,-0.15,0.47,-0.15,0.01,0.03,0.41,0.25,-0.04,0.53,0.06,0.24,0.21
1,2.0,0.62,-0.36,-0.46,0.25,-0.19,-0.55,-0.43,0.07,0.68,0.25,-0.03,0.00,0.43
2,3.0,0.49,-0.60,-0.75,-0.08,-0.20,-0.48,-0.64,-0.44,-0.72,0.10,-0.23,-0.44,0.35
3,4.0,-0.70,-0.25,-0.46,0.06,-0.24,-0.25,-0.66,-0.33,0.45,-0.38,-0.32,-0.23,0.29
4,5.0,-0.36,-0.69,-0.38,0.37,-0.25,0.13,0.66,0.19,-0.11,-0.87,-0.26,0.00,0.16
5,6.0,-0.25,-0.45,0.30,-0.17,-0.18,-0.78,-0.38,-0.57,0.62,0.07,-0.15,-0.21,0.42
6,7.0,0.53,-0.43,0.32,0.07,0.09,0.83,-0.91,-0.75,0.10,0.52,0.12,-0.04,0.16
7,8.0,0.39,-0.53,-0.19,0.06,0.23,0.27,-0.30,0.24,0.16,0.14,-0.01,0.10,0.41
8,9.0,-0.10,0.19,-0.31,-0.06,-0.32,-0.01,-0.49,-0.18,-0.24,0.16,-0.12,-0.15,0.32
9,10.0,0.44,-0.63,0.54,0.37,-0.77,0.70,-0.57,-0.41,-0.14,-0.51,-0.01,-0.19,0.05



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.17,0.06,-0.56,0.18,-0.07,0.03,-0.43,0.16,-0.25,-0.67,-0.04,-0.23,0.19
1,2.0,-0.25,-0.01,-0.41,-0.03,0.11,-0.60,-0.47,0.35,-0.03,-0.22,-0.12,-0.19,0.46
2,3.0,0.20,-0.70,-0.39,0.06,0.54,0.59,0.85,-0.51,-0.44,0.46,-0.06,0.19,0.35
3,4.0,-0.17,-0.10,-0.25,-0.03,0.56,0.34,0.38,0.29,-0.41,-0.00,0.00,0.12,0.42
4,5.0,0.22,0.42,-0.76,0.60,0.62,0.60,-0.88,0.72,-0.35,-0.73,0.22,-0.13,0.24
5,6.0,-0.14,-0.62,-0.22,-0.01,0.24,0.89,0.17,0.23,0.45,-0.67,-0.15,0.21,0.34
6,7.0,-0.14,-0.46,-0.67,0.34,-0.26,0.48,0.01,-0.08,-0.40,-0.46,-0.24,-0.09,0.31
7,8.0,0.33,-0.54,-0.16,-0.84,0.02,0.21,-0.27,0.41,0.46,-0.15,-0.24,0.13,0.53
8,9.0,-0.30,0.70,-0.29,0.70,0.13,-0.63,-0.57,-0.42,0.43,-0.13,0.19,-0.26,0.26
9,10.0,-0.53,-0.42,-0.34,0.36,0.52,-0.19,-0.48,0.06,0.09,-0.39,-0.08,-0.18,0.29



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.33,-0.27,0.63,-0.61,-0.38,0.89,0.48,-0.01,-0.44,0.01,-0.06,0.19,0.15
1,2.0,0.51,0.53,-0.58,-0.04,-0.40,-0.86,-0.23,0.04,-0.21,-0.61,0.00,-0.37,0.03
2,3.0,0.41,-0.91,-0.44,0.22,0.07,0.79,0.30,-0.80,-0.12,0.39,-0.13,0.11,0.46
3,4.0,-0.28,-0.07,0.40,-0.10,0.32,0.93,-0.44,0.21,0.37,0.69,0.05,0.35,0.23
4,5.0,-0.48,-0.16,0.13,0.20,0.20,0.35,0.50,0.39,0.12,-0.33,-0.02,0.21,0.22
5,6.0,-0.33,-0.16,0.39,0.02,0.11,0.43,-0.52,0.23,0.24,-0.90,0.01,-0.10,-0.00
6,7.0,0.18,0.27,0.76,-0.19,0.26,0.48,0.11,0.16,0.81,-0.13,0.26,0.29,0.14
7,8.0,-0.15,0.02,0.73,-0.17,-0.23,0.95,0.41,0.77,0.69,0.36,0.04,0.64,0.62
8,9.0,0.25,-0.63,0.08,-0.66,-0.06,-0.71,-0.86,-0.56,-0.14,-0.83,-0.20,-0.62,-0.04
9,10.0,0.63,0.58,0.89,0.67,-0.16,0.83,-0.90,0.22,0.58,0.56,0.52,0.26,-0.16



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.51,-0.59,-0.45,-0.42,-0.20,0.59,0.76,0.37,-0.64,-0.03,-0.23,0.21,0.46
1,2.0,0.53,-0.18,-0.45,-0.31,-0.59,-0.39,0.03,0.75,-0.86,0.14,-0.20,-0.07,0.15
2,3.0,-0.49,0.35,0.08,0.50,-0.24,-0.03,-0.99,0.21,0.09,-0.26,0.04,-0.20,0.31
3,4.0,-0.37,0.17,0.33,0.47,0.01,0.18,-0.11,-0.03,0.37,-0.00,0.12,0.08,0.67
4,5.0,-0.61,-0.42,-0.82,0.26,0.24,-0.28,-0.95,-0.05,0.10,0.67,-0.27,-0.10,0.20
5,6.0,0.52,-0.03,-0.11,0.01,-0.24,0.50,-0.32,-0.47,-0.31,0.16,0.03,-0.09,0.02
6,7.0,0.06,-0.23,-0.34,0.57,0.38,0.62,-0.22,-0.24,0.02,-0.18,0.09,0.00,0.09
7,8.0,-0.01,0.36,0.32,0.26,-0.62,0.57,-0.03,-0.03,-0.21,-0.30,0.06,-0.00,0.63
8,9.0,0.19,0.86,-0.66,-0.35,0.37,0.38,0.31,-0.37,0.49,-0.85,0.08,-0.01,0.44
9,10.0,0.37,0.24,0.01,0.64,-0.02,-0.56,-0.99,0.41,0.25,-0.11,0.25,-0.20,-0.17


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.12,-0.39,-0.09,0.07,-0.18,-0.01,-0.33,-0.19,0.07,0.00,-0.09,-0.09,0.28
1,ACC,-0.06,-0.17,-0.41,0.13,0.24,0.17,-0.17,0.12,-0.04,-0.30,-0.05,-0.04,0.34
2,AMY,-0.17,-0.12,0.25,0.08,-0.07,0.19,-0.10,-0.07,-0.18,-0.05,-0.01,-0.04,0.29
3,Cau,-0.02,0.24,0.00,0.07,0.08,-0.02,0.15,0.14,-0.36,0.16,0.07,0.01,0.23
4,dlPFC,-0.11,-0.14,0.15,0.05,-0.13,-0.06,0.63,-0.06,-0.09,0.05,-0.04,0.09,0.21
5,INS,0.27,-0.07,0.63,-0.01,-0.05,-0.01,-0.02,0.04,0.23,0.19,0.15,0.09,0.26
6,IPL,0.26,0.02,0.20,-0.19,0.16,0.38,0.84,0.07,-0.32,-0.21,0.09,0.15,0.29
7,mCC,0.00,-0.15,-0.12,0.05,-0.01,-0.38,-0.17,-0.11,0.19,-0.05,-0.05,-0.10,0.29
8,Nac,0.05,0.30,0.07,0.12,0.16,-0.06,0.08,0.12,-0.38,0.08,0.14,-0.03,0.26
9,PCC,-0.31,-0.32,-0.05,0.20,0.48,0.39,-0.19,0.07,0.08,-0.12,0.00,0.05,0.34


#### Model-5

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 2`
- `Number of Dense Layers = 2`
- `hidden units = 32`
- `dropout = 0.3`

In [51]:
HIDDEN_UNITS = 32
DROPOUT_PROB = 0.3
LSTM_LAYERS = 2
DENSE_LAYERS = 2

In [52]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_84"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_105 (LSTM)             (10, 188, 32)             4352      
                                                                 
 lstm_106 (LSTM)             (10, 188, 32)             8320      
                                                                 
 dropout_84 (Dropout)        (10, 188, 32)             0         
                                                                 
 time_distributed_84 (TimeDi  (10, 188, 32)            1056      
 stributed)                                                      
                                                                 
 time_distributed_85 (TimeDi  (10, 188, 1)             33        
 stributed)                                                      
                                                                 
Total params: 13,761
Trainable params: 13,761
Non-tra

In [53]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [54]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [55]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [57]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.12,-0.21,0.49,0.07,-0.05,0.02,0.35,0.05,0.32,0.41,0.08,0.23,0.34
1,2.0,0.51,-0.35,-0.42,0.16,-0.18,-0.63,-0.40,0.05,0.66,0.16,-0.06,-0.03,0.60
2,3.0,0.56,-0.55,-0.57,0.12,0.12,-0.46,-0.57,0.10,-0.75,0.16,-0.06,-0.30,0.30
3,4.0,-0.42,-0.18,-0.38,-0.00,-0.20,-0.20,-0.65,-0.27,0.42,-0.36,-0.24,-0.21,0.34
4,5.0,-0.36,-0.83,-0.29,0.39,-0.12,0.15,0.62,-0.00,0.05,-0.89,-0.24,-0.01,-0.09
5,6.0,-0.36,-0.51,0.25,-0.21,-0.18,-0.72,-0.38,-0.61,0.62,-0.08,-0.20,-0.23,0.49
6,7.0,0.74,-0.48,0.34,0.03,0.23,0.79,-0.93,-0.79,-0.04,0.42,0.17,-0.11,0.20
7,8.0,0.43,-0.50,-0.13,0.10,0.17,0.27,-0.34,0.08,0.01,0.18,0.01,0.04,0.43
8,9.0,0.25,0.03,0.00,-0.03,-0.34,0.04,-0.49,0.20,-0.68,0.33,-0.02,-0.12,0.43
9,10.0,0.55,-0.63,0.46,0.35,-0.73,0.76,-0.60,-0.43,-0.05,-0.38,0.00,-0.14,0.10



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.16,-0.02,-0.52,-0.01,-0.06,-0.07,-0.33,0.15,-0.19,-0.71,-0.09,-0.23,0.44
1,2.0,-0.23,-0.00,-0.44,-0.07,0.08,-0.66,-0.49,0.35,0.00,-0.16,-0.13,-0.19,0.57
2,3.0,0.34,-0.55,-0.51,0.01,0.48,0.60,0.76,-0.44,-0.49,0.50,-0.05,0.19,0.22
3,4.0,-0.08,-0.13,-0.10,-0.04,0.56,0.39,0.34,0.40,-0.42,0.02,0.04,0.15,0.10
4,5.0,0.07,0.46,-0.82,0.56,0.60,0.59,-0.90,0.68,-0.37,-0.75,0.17,-0.15,0.01
5,6.0,-0.17,-0.65,-0.14,-0.02,0.21,0.89,0.08,0.17,0.53,-0.65,-0.15,0.20,0.43
6,7.0,-0.13,-0.44,-0.55,0.23,-0.29,0.42,-0.03,-0.05,-0.36,-0.45,-0.24,-0.09,0.17
7,8.0,0.39,-0.54,-0.11,-0.90,0.03,0.24,-0.32,0.40,0.50,-0.10,-0.23,0.14,0.68
8,9.0,-0.23,0.66,-0.33,0.59,-0.03,-0.69,-0.81,-0.34,0.32,-0.27,0.13,-0.36,0.45
9,10.0,-0.49,-0.45,-0.31,0.34,0.48,-0.21,-0.44,0.03,0.03,-0.43,-0.09,-0.20,0.48



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.26,-0.24,0.55,-0.49,-0.35,0.91,0.53,0.03,-0.46,0.00,-0.05,0.20,0.28
1,2.0,0.50,0.45,-0.52,-0.11,-0.47,-0.88,-0.23,0.05,-0.13,-0.57,-0.03,-0.35,0.08
2,3.0,0.59,-0.74,-0.51,0.15,0.10,0.80,0.37,-0.64,-0.24,0.41,-0.08,0.14,0.28
3,4.0,-0.27,-0.16,0.42,-0.11,0.39,0.94,-0.55,0.23,0.35,0.68,0.05,0.33,0.69
4,5.0,-0.47,-0.13,0.32,0.17,0.35,0.30,0.73,0.38,0.04,-0.32,0.05,0.23,0.20
5,6.0,-0.34,-0.07,0.41,-0.02,0.08,0.44,-0.60,0.21,0.30,-0.83,0.01,-0.10,0.15
6,7.0,0.29,0.22,0.73,-0.15,0.32,0.46,0.07,0.14,0.79,-0.12,0.28,0.27,0.26
7,8.0,-0.16,-0.24,0.62,-0.15,-0.20,0.94,0.36,0.74,0.66,0.28,-0.03,0.60,0.76
8,9.0,0.16,-0.63,0.20,-0.56,-0.15,-0.74,-0.87,-0.50,-0.26,-0.92,-0.20,-0.66,0.04
9,10.0,0.68,0.53,0.81,0.63,-0.04,0.84,-0.90,0.24,0.47,0.56,0.52,0.24,-0.00



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.49,-0.64,-0.39,-0.23,-0.28,0.55,0.75,0.32,-0.64,-0.02,-0.21,0.19,0.53
1,2.0,0.55,-0.23,-0.56,-0.38,-0.49,-0.45,0.04,0.70,-0.84,0.12,-0.22,-0.09,0.22
2,3.0,-0.33,0.28,-0.00,0.54,-0.16,-0.08,-0.99,0.40,0.34,-0.04,0.07,-0.07,0.56
3,4.0,-0.31,0.28,0.21,0.56,-0.09,0.19,-0.07,-0.05,0.47,-0.04,0.13,0.10,0.68
4,5.0,-0.59,-0.32,-0.80,0.30,0.32,-0.41,-0.91,-0.10,0.17,0.63,-0.22,-0.12,0.36
5,6.0,0.55,-0.04,-0.15,0.05,-0.21,0.43,-0.40,-0.58,-0.32,0.07,0.04,-0.16,0.11
6,7.0,0.03,-0.26,-0.34,0.59,0.42,0.57,-0.28,-0.25,-0.06,-0.16,0.09,-0.04,0.19
7,8.0,0.14,-0.03,0.30,0.30,-0.72,0.54,-0.06,-0.02,-0.23,-0.35,-0.00,-0.02,0.86
8,9.0,0.24,0.85,-0.58,-0.32,0.23,0.61,0.17,-0.48,0.56,-0.78,0.08,0.02,0.87
9,10.0,0.38,0.19,0.04,0.61,-0.08,-0.52,-0.97,0.35,0.31,-0.10,0.23,-0.19,-0.00


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.20,-0.42,-0.02,0.10,-0.13,0.00,-0.34,-0.16,0.06,-0.01,-0.05,-0.09,0.31
1,ACC,-0.04,-0.17,-0.38,0.07,0.21,0.15,-0.21,0.13,-0.04,-0.30,-0.06,-0.05,0.36
2,AMY,-0.16,-0.15,0.27,0.06,-0.09,0.14,-0.12,-0.07,-0.26,-0.07,-0.01,-0.07,0.33
3,Cau,-0.03,0.20,0.01,0.04,0.07,-0.04,0.13,0.11,-0.36,0.13,0.06,-0.01,0.27
4,dlPFC,-0.09,-0.17,0.19,0.02,-0.14,-0.09,0.44,-0.08,-0.11,0.05,-0.04,0.04,0.20
5,INS,0.30,-0.08,0.62,0.05,-0.06,-0.03,-0.01,0.06,0.24,0.19,0.17,0.09,0.32
6,IPL,0.29,-0.02,0.16,-0.12,0.10,0.42,0.84,0.18,-0.28,-0.19,0.08,0.19,0.41
7,mCC,-0.01,-0.17,-0.12,0.05,-0.01,-0.38,-0.16,-0.11,0.19,-0.06,-0.05,-0.11,0.36
8,Nac,0.03,0.29,0.07,0.08,0.16,-0.07,0.08,0.11,-0.38,0.06,0.13,-0.04,0.30
9,PCC,-0.30,-0.33,-0.06,0.21,0.48,0.38,-0.24,0.10,0.07,-0.11,0.00,0.04,0.32


#### Model-6

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 2`
- `hidden units = 32`
- `dropout = 0.3`

In [58]:
HIDDEN_UNITS = 32
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 2

In [59]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_105"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_147 (LSTM)             (10, 188, 32)             4352      
                                                                 
 dropout_105 (Dropout)       (10, 188, 32)             0         
                                                                 
 time_distributed_126 (TimeD  (10, 188, 32)            1056      
 istributed)                                                     
                                                                 
 time_distributed_127 (TimeD  (10, 188, 1)             33        
 istributed)                                                     
                                                                 
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________


In [60]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [61]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [62]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [64]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.10,-0.19,0.46,0.37,-0.02,-0.09,0.32,-0.31,0.56,0.38,0.14,0.17,0.30
1,2.0,0.56,-0.16,-0.45,0.11,-0.14,-0.80,-0.55,-0.13,0.46,0.11,-0.02,-0.18,0.57
2,3.0,0.62,-0.39,-0.01,0.63,0.29,-0.45,-0.42,0.64,-0.53,0.24,0.23,-0.10,0.38
3,4.0,-0.23,-0.09,-0.27,-0.04,-0.18,-0.29,-0.68,-0.27,0.46,-0.31,-0.16,-0.22,0.31
4,5.0,-0.12,-0.91,-0.12,0.36,-0.01,0.06,0.59,-0.20,0.24,-0.75,-0.16,-0.01,0.01
5,6.0,-0.40,-0.36,0.19,-0.23,-0.14,-0.62,-0.42,-0.59,0.58,-0.18,-0.19,-0.25,0.48
6,7.0,0.76,-0.50,0.23,0.18,0.39,0.77,-0.97,-0.72,-0.28,0.22,0.21,-0.20,0.20
7,8.0,0.58,-0.42,-0.08,0.15,0.10,0.29,-0.43,-0.04,0.09,0.20,0.07,0.02,0.50
8,9.0,0.43,0.04,0.22,0.05,-0.29,-0.47,-0.49,0.51,-0.76,0.53,0.09,-0.14,0.44
9,10.0,0.60,-0.58,0.48,0.33,-0.65,0.84,-0.68,-0.34,-0.04,-0.31,0.04,-0.11,0.11



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.16,0.02,-0.48,0.20,-0.07,0.00,-0.22,0.17,-0.14,-0.77,-0.03,-0.19,0.34
1,2.0,-0.10,-0.04,-0.33,-0.09,0.01,-0.67,-0.60,0.32,-0.08,-0.09,-0.11,-0.22,0.50
2,3.0,0.52,-0.28,-0.66,0.03,0.44,0.56,0.63,-0.38,-0.22,0.57,0.01,0.23,0.36
3,4.0,-0.06,-0.15,-0.00,-0.09,0.58,0.47,0.17,0.37,-0.47,0.13,0.06,0.13,0.20
4,5.0,-0.06,0.57,-0.80,0.58,0.58,0.55,-0.62,0.61,-0.38,-0.74,0.17,-0.12,0.10
5,6.0,-0.18,-0.61,-0.14,-0.06,0.13,0.91,-0.10,0.19,0.53,-0.63,-0.17,0.18,0.38
6,7.0,-0.18,-0.44,-0.48,0.34,-0.35,0.30,-0.09,-0.14,-0.39,-0.49,-0.22,-0.16,0.20
7,8.0,0.39,-0.43,-0.20,-0.92,0.03,0.32,-0.27,0.39,0.48,-0.08,-0.23,0.17,0.65
8,9.0,-0.05,0.64,-0.26,0.63,-0.04,-0.78,-0.82,-0.11,0.35,-0.17,0.18,-0.31,0.53
9,10.0,-0.49,-0.31,-0.22,0.31,0.45,-0.13,0.09,-0.02,0.04,-0.46,-0.05,-0.10,0.46



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.17,-0.20,0.56,0.06,-0.32,0.89,0.58,0.12,-0.45,-0.01,0.05,0.23,0.41
1,2.0,0.51,0.45,-0.33,-0.06,-0.53,-0.91,-0.23,0.03,-0.28,-0.52,0.01,-0.38,0.14
2,3.0,0.75,-0.41,-0.47,0.12,-0.07,0.78,0.42,-0.22,0.11,0.43,-0.02,0.30,0.33
3,4.0,-0.29,-0.17,0.37,-0.08,0.41,0.92,-0.61,0.26,0.26,0.68,0.05,0.30,0.78
4,5.0,-0.38,-0.10,0.43,0.18,0.37,0.18,0.83,0.50,0.11,-0.27,0.10,0.27,0.25
5,6.0,-0.38,-0.07,0.36,0.01,-0.05,0.39,-0.58,0.15,0.25,-0.73,-0.03,-0.10,0.25
6,7.0,0.37,0.14,0.63,0.06,0.26,0.37,-0.05,0.11,0.76,-0.15,0.29,0.21,0.26
7,8.0,-0.02,-0.13,0.68,-0.17,-0.17,0.95,0.45,0.72,0.64,0.25,0.04,0.60,0.82
8,9.0,0.08,-0.26,0.25,-0.19,-0.18,-0.82,-0.72,-0.19,0.03,-0.96,-0.06,-0.53,0.24
9,10.0,0.70,0.54,0.82,0.63,-0.02,0.85,-0.80,0.35,0.50,0.58,0.53,0.30,0.16



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.45,-0.62,-0.33,-0.13,-0.40,0.52,0.75,0.15,-0.59,0.02,-0.21,0.17,0.52
1,2.0,0.54,-0.20,-0.56,-0.46,-0.33,-0.56,0.05,0.69,-0.73,0.13,-0.20,-0.08,0.22
2,3.0,-0.14,0.35,-0.03,0.56,-0.15,-0.20,-0.95,0.58,0.46,0.20,0.12,0.02,0.67
3,4.0,-0.31,0.32,0.13,0.63,-0.01,0.16,-0.06,0.02,0.42,-0.03,0.15,0.10,0.75
4,5.0,-0.54,-0.14,-0.72,0.33,0.30,-0.54,-0.86,-0.18,0.16,0.59,-0.15,-0.17,0.33
5,6.0,0.53,-0.04,-0.28,0.07,-0.26,0.37,-0.55,-0.54,-0.30,0.11,0.00,-0.18,0.12
6,7.0,-0.02,-0.34,-0.41,0.58,0.39,0.54,-0.40,-0.30,-0.12,-0.15,0.04,-0.09,0.19
7,8.0,0.25,-0.07,0.26,0.32,-0.79,0.50,-0.22,-0.03,-0.32,-0.42,-0.01,-0.10,0.86
8,9.0,0.28,0.86,-0.42,-0.23,0.19,0.54,-0.11,-0.35,0.64,-0.65,0.14,0.01,0.85
9,10.0,0.39,0.42,0.05,0.59,-0.13,-0.45,-0.96,0.32,0.33,-0.11,0.26,-0.17,0.03


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.29,-0.36,0.06,0.19,-0.06,-0.08,-0.37,-0.14,0.08,0.01,0.02,-0.10,0.33
1,ACC,-0.01,-0.10,-0.36,0.09,0.17,0.15,-0.18,0.14,-0.03,-0.27,-0.04,-0.04,0.37
2,AMY,-0.13,-0.11,0.24,0.09,-0.10,0.07,-0.18,-0.05,-0.23,-0.05,-0.00,-0.09,0.35
3,Cau,0.06,0.11,0.09,0.08,0.04,-0.01,0.04,0.16,-0.32,0.17,0.08,0.01,0.31
4,dlPFC,-0.05,-0.09,0.22,0.08,-0.12,-0.17,0.44,0.00,-0.10,0.08,0.01,0.05,0.26
5,INS,0.37,-0.01,0.62,0.17,-0.04,-0.06,-0.02,0.13,0.21,0.22,0.22,0.10,0.33
6,IPL,0.39,0.08,0.24,0.05,0.16,0.49,0.83,0.29,-0.22,-0.04,0.18,0.27,0.47
7,mCC,-0.00,-0.14,-0.14,0.09,-0.03,-0.33,-0.18,-0.09,0.13,-0.05,-0.04,-0.11,0.36
8,Nac,0.10,0.25,0.14,0.13,0.13,-0.05,-0.03,0.14,-0.35,0.05,0.15,-0.05,0.33
9,PCC,-0.23,-0.21,-0.09,0.31,0.48,0.42,-0.25,0.19,0.09,-0.03,0.05,0.08,0.38


#### Model-7

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 4`
- `hidden units = 16`
- `dropout = 0.3`

In [65]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 4

In [66]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_126"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_168 (LSTM)             (10, 188, 16)             1152      
                                                                 
 dropout_126 (Dropout)       (10, 188, 16)             0         
                                                                 
 time_distributed_168 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_169 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_170 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                    

In [67]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [68]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [69]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [71]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.10,-0.18,0.47,0.19,-0.01,-0.03,0.36,-0.26,0.24,0.48,0.11,0.16,0.25
1,2.0,0.61,-0.29,-0.53,0.18,-0.16,-0.72,-0.50,-0.05,0.55,0.18,-0.04,-0.11,0.52
2,3.0,0.55,-0.49,-0.34,0.18,0.03,-0.47,-0.60,0.25,-0.67,0.17,-0.01,-0.26,0.29
3,4.0,-0.43,-0.16,-0.38,-0.01,-0.21,-0.30,-0.67,-0.31,0.47,-0.35,-0.24,-0.23,0.15
4,5.0,-0.30,-0.84,-0.26,0.37,-0.13,0.08,0.63,-0.01,0.08,-0.82,-0.23,-0.01,-0.07
5,6.0,-0.34,-0.41,0.22,-0.21,-0.16,-0.68,-0.41,-0.58,0.61,-0.06,-0.18,-0.22,0.44
6,7.0,0.68,-0.48,0.28,0.14,0.23,0.80,-0.95,-0.77,-0.03,0.40,0.17,-0.11,0.11
7,8.0,0.49,-0.48,-0.15,0.12,0.15,0.27,-0.36,0.15,0.08,0.18,0.03,0.06,0.32
8,9.0,0.14,0.12,-0.12,0.02,-0.29,-0.24,-0.50,0.37,-0.69,0.36,-0.03,-0.14,0.40
9,10.0,0.52,-0.60,0.54,0.35,-0.71,0.78,-0.63,-0.40,-0.11,-0.42,0.02,-0.16,0.11



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.17,0.05,-0.49,0.28,-0.07,0.05,-0.20,0.18,-0.14,-0.79,-0.01,-0.18,0.36
1,2.0,-0.05,-0.05,-0.26,-0.09,-0.01,-0.66,-0.63,0.31,-0.10,-0.07,-0.09,-0.23,0.53
2,3.0,0.57,-0.20,-0.72,0.07,0.42,0.55,0.61,-0.34,-0.09,0.58,0.03,0.26,0.34
3,4.0,-0.08,-0.15,0.02,-0.11,0.58,0.49,0.12,0.33,-0.49,0.18,0.05,0.13,0.07
4,5.0,-0.11,0.62,-0.78,0.58,0.56,0.54,-0.38,0.58,-0.37,-0.72,0.17,-0.07,0.09
5,6.0,-0.18,-0.59,-0.16,-0.08,0.11,0.92,-0.18,0.21,0.53,-0.61,-0.18,0.17,0.43
6,7.0,-0.20,-0.43,-0.47,0.38,-0.37,0.27,-0.10,-0.18,-0.41,-0.51,-0.22,-0.19,0.13
7,8.0,0.41,-0.38,-0.22,-0.91,0.03,0.34,-0.23,0.35,0.49,-0.08,-0.21,0.17,0.69
8,9.0,0.01,0.63,-0.19,0.64,-0.05,-0.73,-0.79,-0.06,0.36,-0.10,0.21,-0.26,0.53
9,10.0,-0.48,-0.26,-0.14,0.30,0.43,-0.09,0.29,-0.05,0.04,-0.47,-0.03,-0.06,0.48



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.18,-0.20,0.57,-0.01,-0.32,0.89,0.57,0.11,-0.47,-0.02,0.04,0.22,0.32
1,2.0,0.51,0.46,-0.33,-0.06,-0.53,-0.91,-0.22,0.02,-0.27,-0.51,0.01,-0.38,0.14
2,3.0,0.75,-0.28,-0.46,0.12,-0.07,0.77,0.40,-0.14,0.19,0.44,0.01,0.33,0.35
3,4.0,-0.30,-0.16,0.37,-0.08,0.40,0.92,-0.60,0.27,0.26,0.67,0.05,0.30,0.79
4,5.0,-0.40,-0.11,0.38,0.18,0.35,0.20,0.80,0.50,0.11,-0.27,0.08,0.27,0.30
5,6.0,-0.38,-0.07,0.35,0.02,-0.06,0.39,-0.57,0.15,0.24,-0.74,-0.03,-0.11,0.29
6,7.0,0.36,0.15,0.64,0.05,0.26,0.37,-0.04,0.11,0.76,-0.14,0.29,0.21,0.31
7,8.0,-0.05,-0.09,0.70,-0.17,-0.18,0.95,0.45,0.73,0.65,0.26,0.04,0.61,0.79
8,9.0,0.10,-0.26,0.23,-0.21,-0.16,-0.82,-0.72,-0.21,0.01,-0.96,-0.06,-0.54,0.24
9,10.0,0.69,0.54,0.83,0.64,-0.05,0.85,-0.81,0.34,0.51,0.57,0.53,0.29,0.08



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.45,-0.59,-0.33,-0.15,-0.40,0.52,0.76,0.12,-0.57,0.03,-0.20,0.17,0.47
1,2.0,0.53,-0.19,-0.55,-0.47,-0.34,-0.58,0.08,0.70,-0.69,0.14,-0.20,-0.07,0.25
2,3.0,-0.20,0.37,-0.00,0.54,-0.19,-0.21,-0.95,0.58,0.43,0.15,0.10,0.00,0.61
3,4.0,-0.30,0.29,0.14,0.62,0.02,0.14,-0.06,-0.02,0.40,-0.00,0.15,0.09,0.73
4,5.0,-0.55,-0.09,-0.71,0.33,0.27,-0.54,-0.86,-0.18,0.12,0.60,-0.15,-0.17,0.25
5,6.0,0.52,-0.04,-0.30,0.07,-0.28,0.38,-0.56,-0.50,-0.31,0.13,-0.01,-0.17,0.13
6,7.0,-0.02,-0.35,-0.43,0.58,0.35,0.56,-0.41,-0.31,-0.11,-0.15,0.03,-0.08,0.17
7,8.0,0.22,-0.01,0.26,0.32,-0.79,0.50,-0.27,-0.03,-0.33,-0.42,0.00,-0.11,0.83
8,9.0,0.27,0.86,-0.46,-0.22,0.22,0.52,-0.13,-0.34,0.64,-0.64,0.13,0.01,0.91
9,10.0,0.39,0.50,0.05,0.59,-0.13,-0.45,-0.95,0.31,0.33,-0.11,0.28,-0.17,-0.04


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.20,-0.38,-0.03,0.13,-0.12,-0.05,-0.36,-0.16,0.05,0.01,-0.04,-0.10,0.25
1,ACC,0.01,-0.08,-0.34,0.11,0.16,0.17,-0.15,0.13,-0.02,-0.26,-0.03,-0.03,0.36
2,AMY,-0.12,-0.12,0.23,0.10,-0.10,0.05,-0.20,-0.05,-0.20,-0.03,-0.00,-0.09,0.38
3,Cau,0.07,0.15,0.09,0.11,0.06,0.01,0.07,0.18,-0.32,0.21,0.10,0.03,0.33
4,dlPFC,-0.05,-0.05,0.20,0.14,-0.09,-0.12,0.52,0.03,-0.08,0.10,0.03,0.09,0.24
5,INS,0.37,-0.01,0.62,0.18,-0.04,-0.08,-0.02,0.13,0.20,0.22,0.22,0.09,0.36
6,IPL,0.40,0.10,0.26,0.06,0.19,0.49,0.83,0.28,-0.22,-0.02,0.20,0.27,0.47
7,mCC,0.01,-0.09,-0.15,0.12,-0.04,-0.30,-0.17,-0.08,0.10,-0.03,-0.03,-0.10,0.38
8,Nac,0.13,0.24,0.16,0.16,0.11,-0.04,-0.07,0.17,-0.33,0.08,0.16,-0.04,0.37
9,PCC,-0.17,-0.11,-0.11,0.34,0.47,0.43,-0.22,0.22,0.10,0.04,0.09,0.11,0.41


#### Model-8

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 3`
- `hidden units = 16`
- `dropout = 0.3`

In [72]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 3

In [73]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_147"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_189 (LSTM)             (10, 188, 16)             1152      
                                                                 
 dropout_147 (Dropout)       (10, 188, 16)             0         
                                                                 
 time_distributed_252 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_253 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_254 (TimeD  (10, 188, 1)             17        
 istributed)                                                     
                                                    

In [74]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [75]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [76]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [78]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.09,-0.17,0.46,0.34,-0.01,-0.08,0.34,-0.37,0.40,0.45,0.14,0.15,0.18
1,2.0,0.61,-0.21,-0.53,0.14,-0.14,-0.80,-0.57,-0.12,0.47,0.14,-0.03,-0.18,0.53
2,3.0,0.60,-0.41,-0.02,0.44,0.15,-0.47,-0.56,0.55,-0.56,0.22,0.15,-0.16,0.31
3,4.0,-0.30,-0.11,-0.32,-0.05,-0.19,-0.33,-0.68,-0.29,0.49,-0.33,-0.19,-0.23,0.08
4,5.0,-0.17,-0.87,-0.17,0.37,-0.05,0.04,0.61,-0.10,0.18,-0.74,-0.18,-0.00,-0.10
5,6.0,-0.38,-0.38,0.19,-0.22,-0.14,-0.63,-0.42,-0.58,0.60,-0.13,-0.19,-0.23,0.45
6,7.0,0.72,-0.50,0.22,0.20,0.33,0.79,-0.96,-0.71,-0.12,0.30,0.19,-0.14,0.08
7,8.0,0.56,-0.44,-0.12,0.15,0.12,0.29,-0.41,0.11,0.11,0.19,0.05,0.06,0.26
8,9.0,0.27,0.11,-0.00,0.05,-0.26,-0.43,-0.49,0.50,-0.73,0.45,0.03,-0.14,0.42
9,10.0,0.54,-0.59,0.53,0.34,-0.66,0.82,-0.67,-0.38,-0.08,-0.38,0.03,-0.14,0.13



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.18,0.06,-0.52,0.31,-0.07,0.07,-0.23,0.18,-0.17,-0.77,-0.01,-0.18,0.38
1,2.0,-0.07,-0.04,-0.26,-0.07,0.01,-0.65,-0.62,0.32,-0.10,-0.10,-0.09,-0.23,0.57
2,3.0,0.54,-0.28,-0.64,0.09,0.48,0.55,0.65,-0.41,-0.17,0.56,0.04,0.24,0.32
3,4.0,-0.12,-0.14,-0.05,-0.08,0.58,0.47,0.16,0.30,-0.48,0.15,0.04,0.12,-0.02
4,5.0,-0.00,0.58,-0.77,0.60,0.58,0.55,-0.56,0.62,-0.36,-0.73,0.20,-0.10,0.04
5,6.0,-0.17,-0.59,-0.18,-0.07,0.13,0.92,-0.11,0.22,0.51,-0.62,-0.18,0.18,0.47
6,7.0,-0.20,-0.44,-0.53,0.41,-0.35,0.31,-0.09,-0.17,-0.42,-0.51,-0.22,-0.18,0.08
7,8.0,0.41,-0.40,-0.21,-0.90,0.02,0.32,-0.23,0.35,0.49,-0.09,-0.22,0.17,0.70
8,9.0,-0.05,0.65,-0.19,0.68,0.01,-0.72,-0.82,-0.10,0.39,-0.06,0.22,-0.26,0.50
9,10.0,-0.51,-0.28,-0.16,0.31,0.45,-0.09,0.17,-0.03,0.05,-0.45,-0.04,-0.07,0.49



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.30,-0.26,0.66,-0.18,-0.36,0.88,0.51,0.06,-0.36,0.11,0.03,0.24,0.26
1,2.0,0.52,0.55,-0.52,0.00,-0.44,-0.89,-0.23,0.03,-0.33,-0.61,0.02,-0.41,0.18
2,3.0,0.58,-0.94,-0.43,0.18,0.01,0.78,0.34,-0.61,0.06,0.40,-0.12,0.19,0.37
3,4.0,-0.30,-0.05,0.38,-0.09,0.33,0.92,-0.49,0.19,0.33,0.72,0.05,0.33,0.71
4,5.0,-0.45,-0.17,0.17,0.23,0.21,0.31,0.71,0.44,0.23,-0.35,-0.00,0.27,0.27
5,6.0,-0.35,-0.18,0.38,0.01,0.07,0.41,-0.50,0.23,0.19,-0.89,-0.01,-0.11,0.23
6,7.0,0.22,0.24,0.73,-0.07,0.25,0.45,0.07,0.13,0.79,-0.18,0.27,0.25,0.21
7,8.0,-0.03,0.10,0.76,-0.20,-0.21,0.95,0.46,0.77,0.67,0.40,0.08,0.65,0.75
8,9.0,0.23,-0.49,0.14,-0.47,-0.08,-0.75,-0.77,-0.37,-0.00,-0.79,-0.13,-0.54,0.09
9,10.0,0.65,0.60,0.91,0.64,-0.14,0.84,-0.87,0.26,0.65,0.61,0.53,0.30,-0.05



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.44,-0.61,-0.31,-0.04,-0.45,0.51,0.74,0.13,-0.58,0.04,-0.19,0.17,0.56
1,2.0,0.55,-0.20,-0.62,-0.49,-0.24,-0.58,0.07,0.67,-0.70,0.09,-0.20,-0.09,0.25
2,3.0,-0.14,0.29,-0.04,0.56,-0.15,-0.19,-0.94,0.63,0.44,0.32,0.10,0.05,0.57
3,4.0,-0.29,0.32,0.11,0.65,-0.08,0.17,-0.04,-0.02,0.40,-0.01,0.14,0.10,0.72
4,5.0,-0.54,0.02,-0.71,0.37,0.35,-0.60,-0.86,-0.18,0.14,0.58,-0.10,-0.18,0.40
5,6.0,0.53,-0.04,-0.27,0.09,-0.27,0.36,-0.59,-0.57,-0.34,0.05,0.01,-0.22,0.13
6,7.0,-0.04,-0.36,-0.39,0.59,0.38,0.52,-0.43,-0.30,-0.12,-0.14,0.04,-0.09,0.21
7,8.0,0.25,-0.25,0.27,0.31,-0.80,0.50,-0.24,0.02,-0.32,-0.43,-0.04,-0.09,0.87
8,9.0,0.27,0.86,-0.49,-0.23,0.19,0.55,-0.11,-0.43,0.64,-0.59,0.12,0.01,0.92
9,10.0,0.39,0.34,0.06,0.57,-0.15,-0.44,-0.95,0.30,0.35,-0.09,0.24,-0.17,0.07


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.25,-0.36,0.02,0.18,-0.09,-0.08,-0.38,-0.14,0.08,0.02,0.00,-0.10,0.24
1,ACC,-0.00,-0.09,-0.35,0.13,0.18,0.17,-0.17,0.13,-0.03,-0.26,-0.03,-0.03,0.35
2,AMY,-0.14,-0.11,0.25,0.09,-0.09,0.12,-0.15,-0.05,-0.23,-0.04,0.00,-0.07,0.31
3,Cau,0.11,0.08,0.14,0.11,0.04,0.01,-0.00,0.20,-0.28,0.23,0.10,0.03,0.36
4,dlPFC,-0.08,-0.08,0.18,0.10,-0.10,-0.11,0.56,0.01,-0.09,0.08,0.01,0.09,0.21
5,INS,0.35,0.01,0.63,0.15,-0.03,-0.05,0.00,0.14,0.22,0.23,0.22,0.11,0.29
6,IPL,0.39,0.10,0.26,0.00,0.21,0.46,0.84,0.22,-0.25,-0.05,0.19,0.24,0.46
7,mCC,-0.01,-0.10,-0.14,0.08,-0.02,-0.37,-0.15,-0.10,0.16,-0.03,-0.04,-0.10,0.34
8,Nac,0.11,0.26,0.13,0.16,0.13,-0.04,-0.02,0.16,-0.35,0.09,0.16,-0.03,0.36
9,PCC,-0.24,-0.20,-0.08,0.32,0.48,0.42,-0.24,0.18,0.09,-0.03,0.06,0.08,0.36


#### Model-9

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 2`
- `hidden units = 16`
- `dropout = 0.3`

In [79]:
HIDDEN_UNITS = 16
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 2

In [80]:
sample_model = avg_bold_model()
sample_model.build(input_shape=batch_x.shape)
sample_model.summary()

Model: "sequential_168"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_210 (LSTM)             (10, 188, 16)             1152      
                                                                 
 dropout_168 (Dropout)       (10, 188, 16)             0         
                                                                 
 time_distributed_315 (TimeD  (10, 188, 16)            272       
 istributed)                                                     
                                                                 
 time_distributed_316 (TimeD  (10, 188, 1)             17        
 istributed)                                                     
                                                                 
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


In [81]:
models = [avg_bold_model() for _ in range(len(ROI_names))]

In [82]:
for i in range(len(ROI_names)):
  models[i].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [83]:
for i in range(len(ROI_names)):
  models[i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [models[i].predict(next(test_data_generator[i].generate())[0]) for i in range(len(ROI_names))]

In [85]:
generate_each_sub_n_overall_pred_corr(preds)

***Each Test Subject's Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***


***dmPFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.11,-0.19,0.48,0.17,-0.02,-0.02,0.36,-0.19,0.30,0.44,0.11,0.18,0.27
1,2.0,0.57,-0.32,-0.48,0.16,-0.16,-0.69,-0.48,-0.02,0.60,0.17,-0.05,-0.08,0.49
2,3.0,0.58,-0.49,-0.30,0.37,0.04,-0.47,-0.61,0.40,-0.62,0.18,0.04,-0.22,0.35
3,4.0,-0.42,-0.17,-0.37,-0.01,-0.20,-0.26,-0.67,-0.30,0.46,-0.35,-0.23,-0.22,0.29
4,5.0,-0.28,-0.83,-0.24,0.36,-0.13,0.09,0.63,-0.00,0.10,-0.83,-0.22,-0.00,0.07
5,6.0,-0.37,-0.47,0.21,-0.22,-0.16,-0.70,-0.40,-0.59,0.62,-0.08,-0.20,-0.23,0.42
6,7.0,0.71,-0.48,0.30,0.12,0.25,0.80,-0.95,-0.78,-0.04,0.39,0.18,-0.12,0.17
7,8.0,0.51,-0.48,-0.13,0.12,0.17,0.28,-0.37,0.11,0.11,0.18,0.04,0.06,0.50
8,9.0,0.21,0.08,-0.08,-0.00,-0.30,-0.12,-0.49,0.34,-0.70,0.37,-0.02,-0.12,0.37
9,10.0,0.52,-0.62,0.51,0.34,-0.71,0.76,-0.63,-0.44,-0.06,-0.40,0.01,-0.15,0.08



***ACC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.17,0.07,-0.55,0.26,-0.07,0.05,-0.31,0.17,-0.21,-0.73,-0.02,-0.21,0.24
1,2.0,-0.15,-0.02,-0.33,-0.06,0.05,-0.64,-0.55,0.33,-0.07,-0.16,-0.10,-0.22,0.44
2,3.0,0.41,-0.49,-0.51,0.08,0.52,0.58,0.74,-0.48,-0.32,0.51,0.00,0.21,0.37
3,4.0,-0.14,-0.11,-0.15,-0.06,0.57,0.42,0.28,0.28,-0.45,0.09,0.02,0.12,0.00
4,5.0,0.12,0.51,-0.77,0.60,0.60,0.58,-0.82,0.68,-0.34,-0.73,0.21,-0.13,0.02
5,6.0,-0.16,-0.62,-0.20,-0.04,0.18,0.90,0.04,0.23,0.48,-0.66,-0.17,0.20,0.33
6,7.0,-0.17,-0.44,-0.61,0.38,-0.32,0.38,-0.04,-0.15,-0.42,-0.49,-0.23,-0.14,0.06
7,8.0,0.35,-0.47,-0.20,-0.88,0.02,0.27,-0.26,0.39,0.45,-0.12,-0.24,0.15,0.59
8,9.0,-0.18,0.68,-0.25,0.71,0.08,-0.73,-0.83,-0.25,0.41,-0.12,0.21,-0.30,0.46
9,10.0,-0.53,-0.33,-0.27,0.34,0.49,-0.14,-0.21,0.02,0.08,-0.42,-0.06,-0.13,0.38



***AMY***


...

,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.23,-0.22,0.58,-0.34,-0.34,0.89,0.54,0.07,-0.47,-0.04,-0.02,0.20,0.18
1,2.0,0.51,0.49,-0.46,-0.06,-0.48,-0.91,-0.23,0.03,-0.24,-0.56,0.00,-0.38,0.14
2,3.0,0.63,-0.81,-0.45,0.20,-0.02,0.78,0.33,-0.58,0.03,0.41,-0.09,0.19,0.36
3,4.0,-0.29,-0.14,0.39,-0.09,0.37,0.93,-0.54,0.25,0.31,0.68,0.05,0.33,0.65
4,5.0,-0.45,-0.13,0.29,0.17,0.30,0.26,0.70,0.47,0.09,-0.28,0.04,0.25,0.25
5,6.0,-0.36,-0.11,0.37,0.01,0.01,0.40,-0.56,0.19,0.25,-0.83,-0.02,-0.11,0.19
6,7.0,0.29,0.20,0.71,-0.06,0.26,0.42,0.02,0.14,0.79,-0.14,0.28,0.25,0.23
7,8.0,-0.17,-0.07,0.70,-0.16,-0.20,0.95,0.43,0.74,0.67,0.28,0.02,0.61,0.71
8,9.0,0.16,-0.43,0.19,-0.43,-0.13,-0.78,-0.78,-0.38,-0.13,-0.95,-0.13,-0.60,0.12
9,10.0,0.67,0.55,0.84,0.66,-0.09,0.84,-0.85,0.32,0.52,0.56,0.53,0.28,-0.05



***lOFC***


,Subs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,1.0,0.47,-0.62,-0.37,-0.27,-0.29,0.55,0.76,0.24,-0.61,-0.00,-0.22,0.19,0.46
1,2.0,0.54,-0.20,-0.54,-0.40,-0.48,-0.49,0.06,0.71,-0.78,0.14,-0.22,-0.07,0.21
2,3.0,-0.30,0.38,0.06,0.55,-0.26,-0.13,-0.99,0.43,0.36,-0.07,0.09,-0.08,0.64
3,4.0,-0.37,0.23,0.26,0.54,0.15,0.18,-0.11,0.01,0.38,-0.01,0.16,0.09,0.76
4,5.0,-0.57,-0.26,-0.77,0.30,0.28,-0.41,-0.89,-0.13,0.13,0.63,-0.20,-0.13,0.19
5,6.0,0.54,-0.04,-0.23,0.04,-0.27,0.43,-0.42,-0.48,-0.33,0.18,0.01,-0.12,0.10
6,7.0,0.02,-0.29,-0.40,0.58,0.38,0.59,-0.31,-0.27,-0.07,-0.17,0.06,-0.05,0.19
7,8.0,0.14,0.10,0.29,0.30,-0.73,0.53,-0.15,-0.01,-0.27,-0.37,0.02,-0.05,0.79
8,9.0,0.24,0.86,-0.55,-0.29,0.26,0.55,0.04,-0.39,0.59,-0.76,0.10,0.01,0.80
9,10.0,0.38,0.31,0.04,0.61,-0.09,-0.51,-0.98,0.35,0.31,-0.10,0.25,-0.19,-0.11


***Average across test subjects Correlation Coefficient between predicted and actual average BOLD values of each Ascent and Descent for ROIs***



,ROIs,A1,A2,A3,A4,A5,D1,D2,D3,D4,D5,Avg_Asc,Avg_Des,Overall
0,dmPFC,0.21,-0.40,-0.01,0.14,-0.12,-0.03,-0.36,-0.15,0.08,0.01,-0.03,-0.09,0.30
1,ACC,-0.03,-0.12,-0.38,0.13,0.21,0.17,-0.20,0.12,-0.04,-0.28,-0.04,-0.05,0.29
2,AMY,-0.14,-0.12,0.25,0.08,-0.09,0.13,-0.14,-0.05,-0.24,-0.05,-0.00,-0.07,0.29
3,Cau,0.02,0.18,0.05,0.09,0.08,-0.00,0.12,0.16,-0.34,0.16,0.08,0.02,0.28
4,dlPFC,-0.09,-0.09,0.14,0.11,-0.11,-0.06,0.64,-0.03,-0.08,0.08,-0.01,0.11,0.13
5,INS,0.34,-0.01,0.63,0.12,-0.03,-0.04,-0.01,0.12,0.23,0.22,0.21,0.10,0.26
6,IPL,0.36,0.07,0.24,-0.05,0.17,0.45,0.84,0.21,-0.27,-0.09,0.16,0.23,0.41
7,mCC,0.00,-0.18,-0.14,0.10,-0.03,-0.33,-0.19,-0.09,0.13,-0.07,-0.05,-0.11,0.36
8,Nac,0.04,0.31,0.03,0.14,0.16,-0.05,0.10,0.13,-0.38,0.13,0.14,-0.02,0.27
9,PCC,-0.30,-0.26,-0.06,0.31,0.49,0.41,-0.20,0.10,0.08,-0.09,0.04,0.06,0.35
